In [2]:

import numpy as np 
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import xgboost as xgb
import lightgbm as lgb
import catboost
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.gridspec as gridspec
%matplotlib inline

import lightgbm as lgb
from sklearn import metrics
import xgboost as xgb
import catboost as cb

import warnings
warnings.filterwarnings("ignore")

import gc
gc.enable()

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
train_identity = pd.read_csv('train_identity.csv')
train_transaction = pd.read_csv('train_transaction.csv')
test_identity = pd.read_csv('test_identity.csv')
test_transaction = pd.read_csv('test_transaction.csv')
sub = pd.read_csv('sample_submission.csv')

train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [10]:
print('train_transaction has shape',train_transaction.shape)
print('train_identity has shape',train_identity.shape)
print('test_transaction has shape',test_transaction.shape)
print('test_identity has shape',test_identity.shape)

train_transaction has shape (590540, 394)
train_identity has shape (144233, 41)
test_transaction has shape (506691, 393)
test_identity has shape (141907, 41)


In [11]:
del train_identity,train_transaction,test_identity,test_transaction
gc.collect()

91

In [12]:
def red_mem_usage(df, verbose=True):
    num = ['int16','int32','int64','float16','float32','float64']
    start_mem = df.memory_usage(deep = True).sum()/1024**2
    for col in df.columns:
        col_types = df[col].dtypes
        if col_types in num:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_types)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem=df.memory_usage(deep = True).sum()/1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    
    

# Feature Engineering

In [13]:
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score

In [14]:
k = []
for i in range(1,39):
    if i<10:
        test = test.rename(columns={"id-0"+str(i): 'id_0' + str(i)})

    else:
        test = test.rename(columns={"id-"+str(i): 'id_' + str(i)})


In [15]:
def id_split(dataframe, d=1):
    dataframe['device_name'] = dataframe['DeviceInfo'].str.split('/', expand=True)[0]
    dataframe['device_version'] = dataframe['DeviceInfo'].str.split('/', expand=True)[1]
    if d==1:
        dataframe['OS_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[0]
        dataframe['version_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[1]

        dataframe['browser_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[0]
        dataframe['version_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[1]

        dataframe['screen_width'] = dataframe['id_33'].str.split('x', expand=True)[0]
        dataframe['screen_height'] = dataframe['id_33'].str.split('x', expand=True)[1]
        
    else:
        dataframe['OS_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[0]
        dataframe['version_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[1]

        dataframe['browser_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[0]
        dataframe['version_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[1]

        dataframe['screen_width'] = dataframe['id_33'].str.split('x', expand=True)[0]
        dataframe['screen_height'] = dataframe['id_33'].str.split('x', expand=True)[1]
        
        dataframe['id_34'] = dataframe['id_34'].str.split(':', expand=True)[1]
        dataframe['id_23'] = dataframe['id_23'].str.split(':', expand=True)[1]
   

    dataframe.loc[dataframe['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
    dataframe.loc[dataframe['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
    dataframe.loc[dataframe['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
    dataframe.loc[dataframe['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
    dataframe.loc[dataframe['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
    dataframe.loc[dataframe['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

    dataframe.loc[dataframe.device_name.isin(dataframe.device_name.value_counts()[dataframe.device_name.value_counts() < 200].index), 'device_name'] = "Others"
    dataframe['had_id'] = 1
    gc.collect()
    
    return dataframe

In [16]:
train = id_split(train,d=1)
test = id_split(test,2)

In [17]:
print(train.shape,test.shape)

(590540, 443) (506691, 442)


In [18]:
useful_features = ['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1',
                   'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13',
                   'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M2', 'M3',
                   'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V17',
                   'V19', 'V20', 'V29', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V40', 'V44', 'V45', 'V46', 'V47', 'V48',
                   'V49', 'V51', 'V52', 'V53', 'V54', 'V56', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71',
                   'V72', 'V73', 'V74', 'V75', 'V76', 'V78', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V87', 'V90', 'V91', 'V92',
                   'V93', 'V94', 'V95', 'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130', 'V131', 'V138', 'V139', 'V140',
                   'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V154', 'V156', 'V158', 'V159', 'V160', 'V161',
                   'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V169', 'V170', 'V171', 'V172', 'V173', 'V175', 'V176', 'V177',
                   'V178', 'V180', 'V182', 'V184', 'V187', 'V188', 'V189', 'V195', 'V197', 'V200', 'V201', 'V202', 'V203', 'V204',
                   'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V219', 'V220',
                   'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V231', 'V233', 'V234', 'V238', 'V239',
                   'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V249', 'V251', 'V253', 'V256', 'V257', 'V258', 'V259', 'V261',
                   'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276',
                   'V277', 'V278', 'V279', 'V280', 'V282', 'V283', 'V285', 'V287', 'V288', 'V289', 'V291', 'V292', 'V294', 'V303',
                   'V304', 'V306', 'V307', 'V308', 'V310', 'V312', 'V313', 'V314', 'V315', 'V317', 'V322', 'V323', 'V324', 'V326',
                   'V329', 'V331', 'V332', 'V333', 'V335', 'V336', 'V338', 'id_01', 'id_02', 'id_03', 'id_05', 'id_06', 'id_09',
                   'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31', 'id_32', 'id_33',
                   'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'device_name', 'device_version', 'OS_id_30', 'version_id_30',
                   'browser_id_31', 'version_id_31', 'screen_width', 'screen_height', 'had_id']

In [19]:
cols_to_drop = [col for col in train.columns if col not in useful_features]
cols_to_drop.remove('isFraud')
cols_to_drop.remove('TransactionDT')

train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [20]:
print(train.shape,test.shape)

(590540, 290) (506691, 289)


In [21]:
from sklearn.preprocessing import LabelEncoder

In [22]:
# New feature - log of transaction amount.
train['TransactionAmt_Log'] = np.log(train['TransactionAmt'])
test['TransactionAmt_Log'] = np.log(test['TransactionAmt'])

#seeing if feature addr2==87 or not
train['USA_or_not']= (train['addr2']==87).astype('int')
test['USA_or_not']= (test['addr2']==87).astype('int')

# New feature - decimal part of the transaction amount.
train['TransactionAmt_decimal'] = ((train['TransactionAmt'] - train['TransactionAmt'].astype(int)) * 1000).astype(int)
test['TransactionAmt_decimal'] = ((test['TransactionAmt'] - test['TransactionAmt'].astype(int)) * 1000).astype(int)

# New feature - hour of the day in which a transaction happened.
train['Transaction_hour'] = np.floor(train['TransactionDT'] / 3600) % 24
test['Transaction_hour'] = np.floor(test['TransactionDT'] / 3600) % 24

# New feature - day of week in which a transaction happened.
train['Transaction_day_of_week'] = np.floor((train['TransactionDT'] / (3600 * 24) - 1) % 7)
test['Transaction_day_of_week'] = np.floor((test['TransactionDT'] / (3600 * 24) - 1) % 7)


In [23]:
# Taken from https://www.kaggle.com/davidcairuz/feature-engineering-lightgbm

# Some arbitrary features interaction
for feature in ['id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 
                'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1']:

    f1, f2 = feature.split('__')
    train[feature] = train[f1].astype(str) + '_' + train[f2].astype(str)
    test[feature] = test[f1].astype(str) + '_' + test[f2].astype(str)

    le = LabelEncoder()
    le.fit(list(train[feature].astype(str).values) + list(test[feature].astype(str).values))
    train[feature] = le.transform(list(train[feature].astype(str).values))
    test[feature] = le.transform(list(test[feature].astype(str).values))

# Encoding - count encoding for both train and test
for feature in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'id_36']:
    train[feature + '_count_full'] = train[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))
    test[feature + '_count_full'] = test[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))

# Encoding - count encoding separately for train and test
for feature in ['id_01', 'id_31', 'id_33', 'id_36']:
    train[feature + '_count_dist'] = train[feature].map(train[feature].value_counts(dropna=False))
    test[feature + '_count_dist'] = test[feature].map(test[feature].value_counts(dropna=False))

In [24]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo', 'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink', 'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other', 'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo', 'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other', 'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft', 'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other', 'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']

In [25]:
# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499

for c in ['P_emaildomain', 'R_emaildomain']:
    train[c + '_bin'] = train[c].map(emails)
    test[c + '_bin'] = test[c].map(emails)
    
    train[c + '_suffix'] = train[c].map(lambda x: str(x).split('.')[-1])
    test[c + '_suffix'] = test[c].map(lambda x: str(x).split('.')[-1])
    
    train[c + '_suffix'] = train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test[c + '_suffix'] = test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

In [26]:
cat_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        cat_col.append(col)
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))

In [27]:
cat_col.append('addr1')
cat_col.append('addr2')
cat_col

['ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'id_12',
 'id_15',
 'id_30',
 'id_31',
 'id_33',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo',
 'device_name',
 'device_version',
 'OS_id_30',
 'version_id_30',
 'browser_id_31',
 'version_id_31',
 'screen_width',
 'screen_height',
 'P_emaildomain_bin',
 'P_emaildomain_suffix',
 'R_emaildomain_bin',
 'R_emaildomain_suffix',
 'addr1',
 'addr2']

In [28]:
%%time
red_mem_usage(train)
red_mem_usage(test)

Mem. usage decreased to 397.61 Mb (72.4% reduction)
Mem. usage decreased to 348.40 Mb (71.7% reduction)
Wall time: 2min 21s


,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M2,M3,M4,M5,M6,M7,M8,M9,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V17,V19,V20,V29,V30,V33,V34,V35,V36,V37,V38,V40,V44,V45,V46,V47,V48,V49,V51,V52,V53,V54,V56,V58,V59,V60,V61,V62,V63,V64,V69,V70,V71,V72,V73,V74,V75,V76,V78,V80,V81,V82,V83,V84,V85,V87,V90,V91,V92,V93,V94,V95,V96,V97,V99,V100,V126,V127,V128,V130,V131,V138,V139,V140,V143,V145,V146,V147,V149,V150,V151,V152,V154,V156,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V169,V170,V171,V172,V173,V175,V176,V177,V178,V180,V182,V184,V187,V188,V189,V195,V197,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V212,V213,V214,V215,V216,V217,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V231,V233,V234,V238,V239,V242,V243,V244,V245,V246,V247,V249,V251,V253,V256,V257,V258,V259,V261,V262,V263,V264,V265,V266,V267,V268,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V282,V283,V285,V287,V288,V289,V291,V292,V294,V303,V304,V306,V307,V308,V310,V312,V313,V314,V315,V317,V322,V323,V324,V326,V329,V331,V332,V333,V335,V336,V338,id_01,id_02,id_03,id_05,id_06,id_09,id_11,id_12,id_13,id_14,id_15,id_17,id_19,id_20,id_30,id_31,id_32,id_33,id_36,id_37,id_38,DeviceType,DeviceInfo,device_name,device_version,OS_id_30,version_id_30,browser_id_31,version_id_31,screen_width,screen_height,had_id,TransactionAmt_Log,USA_or_not,TransactionAmt_decimal,Transaction_hour,Transaction_day_of_week,id_02__id_20,id_02__D8,D11__DeviceInfo,DeviceInfo__P_emaildomain,P_emaildomain__C2,card2__dist1,card1__card5,card2__id_20,card5__P_emaildomain,addr1__card1,card1_count_full,card2_count_full,card3_count_full,card4_count_full,card5_count_full,card6_count_full,id_36_count_full,id_01_count_dist,id_31_count_dist,id_33_count_dist,id_36_count_dist,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix
0,18403224,31.953125,4,10409,111.0,150.0,4,226.0,2,170.0,87.0,1.0,16,32,6.0,6.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,418.0,203.0,NaN,NaN,NaN,409.0,1,0,3,2,0,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.000000,47.950001,0.000000,47.950001,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.000000,47.950001,0.000000,47.950001,0.000000,0.00000,0.00000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,3,NaN,NaN,NaN,86,136,NaN,461,2,2,2,2,2740,13,380,6,49,31,43,247,216,1,3.464844,1,949,0.0,2.0,270749,259912,124,8507,2401,807,482,372,1375,5275,150,82537,956845,719649,553537,824959,819269,364784,370066,436020,369714,4,0,6,6
1,18403263,49.000000,4,4272,111.0,150.0,4,226.0,2,299.0,87.0,4.0,2,32,3.0,2.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,NaN,NaN,NaN,231.0,634.0,NaN,NaN,NaN,634.0,0,0,0,2,0,2,2,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,4.0,1.0,0.000000,280.000000,77.000000,280.000000,77.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [29]:
X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT'], axis=1)
y = train.sort_values('TransactionDT')['isFraud']

X_test = test.drop(['TransactionDT'], axis=1)


In [31]:
print(X.shape, X_test.shape)

(590540, 318) (506691, 318)


# ALGORITHMS

## For Validation

In [32]:
# 4 months training
X_train = X[:int(590540*(2/3))].copy()
y_train = y[:int(590540*(2/3))].copy()

# 1 month validation with 1 month inbetween
X_val = X[int(590540*(1/6)):-1].copy()
y_val = y[int(590540*(1/6)):-1].copy()

In [33]:
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_val,m.predict_proba(test)[:,1]))

## LightGBM

In [ ]:
lg = lgb.LGBMClassifier()
lg.fit(X_train,y_train,categorical_feature = cat_col)

# Without Categorical Features
auc(lg, X_train, X_val)

## CatBoost

In [ ]:
for header in cat_col:
    X_train[header] = X_train[header].astype('category').cat.codes
    X_val[header] = X_val[header].astype('category').cat.codes
    
#With Categorical features
clf = cb.CatBoostClassifier()
cat_dims = [X_train.columns.get_loc(i) for i in cat_col[:-1]] 
clf.fit(X_train,y_train, cat_features = cat_dims)
auc_tr_cb,auc_test_cb = auc(clf, X_train, X_val)
print(auc_tr_cb,auc_test_cb)

## XGBoost

In [1]:
model = xgb.XGBClassifier()
param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
              "n_estimators": [200],
              "learning_rate": [0.05, 0.1,0.16],}
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(train, y_train)

grid_search.best_estimator_




model = xgb.XGBClassifier()
model.fit(X_train, y_train)
auc(model, X_train, X_val)

NameError: name 'xgb' is not defined

## Submissions

In [37]:
def submission(model, name = "submission.csv",category_features = []):
    
    X_mod = X.copy()
    y_mod = y.copy()
    X_mod_test = X_test.copy()
    pred = np.zeros(X_mod_test.shape[0])
    
    if model == 'lgbm':
        lg = lgb.LGBMClassifier()
        lg.fit(X_mod,y_mod,feature_name='auto', categorical_feature = category_features)
        pred = lg.predict_proba(X_mod_test)[:,1]
    elif model == 'catboost':
        for header in category_features:
            X_mod[header] = X_mod[header].astype('category').cat.codes
            X_mod_test[header] = X_mod_test[header].astype('category').cat.codes
        clf = cb.CatBoostClassifier()
        cat_dims = [X_train.columns.get_loc(i) for i in category_features[:-1]] 
        clf.fit(X_mod,y_mod, cat_features = cat_dims)
        pred = clf.predict_proba(X_mod_test)[:,1]
    elif model == 'xgboost':
        model_xgb = xgb.XGBClassifier()
        model_xgb.fit(X_mod, y_mod)
        pred = model_xgb.predict_proba(X_mod_test)[:,1]
    sub['isFraud'] = pred
    sub.to_csv(name, index=False)

In [ ]:
X_test.shape

### 1. Experiment

In [ ]:
%%time
submission(model='xgboost',name = 'sub_xgb_1')

In [38]:
%%time
submission(model='lgbm',name='sub_lgbm_1')

(506691,)         TransactionAmt  ProductCD  card1  card2  card3  card4  card5  card6  \
0            31.953125          4  10409  111.0  150.0      4  226.0      2   
1            49.000000          4   4272  111.0  150.0      4  226.0      2   
2           171.000000          4   4476  574.0  150.0      4  226.0      2   
3           285.000000          4  10989  360.0  150.0      4  166.0      2   
4            67.937500          4  18018  452.0  150.0      2  117.0      2   
...                ...        ...    ...    ...    ...    ...    ...    ...   
506686       94.687500          0  13832  375.0  185.0      2  224.0      2   
506687       12.171875          0   3154  408.0  185.0      2  224.0      2   
506688       49.000000          4  16661  490.0  150.0      4  226.0      2   
506689      202.000000          4  16621  516.0  150.0      2  224.0      2   
506690       24.343750          0   5713  168.0  144.0      4  147.0      1   

        addr1  addr2   dist1  P_emaildoma

[506691 rows x 318 columns]
Wall time: 28.3 s


In [ ]:
%%time
submission(model='catboost',name='sub_cb_1')

### 2. Experiment

In [39]:
%%time
submission(model='lgbm',name='sub_lgbm_2',category_features=cat_col)

(506691,)         TransactionAmt  ProductCD  card1  card2  card3  card4  card5  card6  \
0            31.953125          4  10409  111.0  150.0      4  226.0      2   
1            49.000000          4   4272  111.0  150.0      4  226.0      2   
2           171.000000          4   4476  574.0  150.0      4  226.0      2   
3           285.000000          4  10989  360.0  150.0      4  166.0      2   
4            67.937500          4  18018  452.0  150.0      2  117.0      2   
...                ...        ...    ...    ...    ...    ...    ...    ...   
506686       94.687500          0  13832  375.0  185.0      2  224.0      2   
506687       12.171875          0   3154  408.0  185.0      2  224.0      2   
506688       49.000000          4  16661  490.0  150.0      4  226.0      2   
506689      202.000000          4  16621  516.0  150.0      2  224.0      2   
506690       24.343750          0   5713  168.0  144.0      4  147.0      1   

        addr1  addr2   dist1  P_emaildoma

[506691 rows x 318 columns]
Wall time: 31.1 s


In [ ]:
%%time
submission(model='catboost',name='sub_cb_2',category_features=cat_col)

### 3. Experiment

In [34]:
def submission2(model, name = "submission.csv",category_features = []):
    
    X_mod = X.copy()
    y_mod = y.copy()
    X_mod_test = X_test.copy()
    pred = np.zeros(X_mod_test.shape[0])
    
    if model == 'lgbm':
        lg = lgb.LGBMClassifier(max_depth = 15, num_iterations = 200, num_leaves = 100,
                            subsample = 0.8, init_score = 0.3, boosting_type = 'goss',
                            objective = 'binary', min_split_gain =0.3)
        lg.fit(X_mod,y_mod,feature_name='auto', categorical_feature = category_features)
        pred = lg.predict_proba(X_mod_test)[:,1]
    elif model == 'catboost':
        for header in category_features:
            X_mod[header] = X_mod[header].astype('category').cat.codes
            X_mod_test[header] = X_mod_test[header].astype('category').cat.codes
        clf = cb.CatBoostClassifier(learning_rate = 0.1, depth = 15, l2_leaf_reg = 3.5,
                            rsm = 0.8, subsample = 0.8, iterations = 100)
        cat_dims = [X_train.columns.get_loc(i) for i in category_features[:-1]] 
        clf.fit(X_mod,y_mod, cat_features = cat_dims)
        pred = clf.predict_proba(X_mod_test)[:,1]
    elif model == 'xgboost':
        model_xgb = xgb.XGBClassifier(base_score = 0.3, learning_rate = 0.1, max_depth = 15, 
                              colsample_bytree = 0.8, subsample = 0.8, n_estimators = 100,
                              gamma = 0.3)
        model_xgb.fit(X_mod, y_mod)
        pred = model_xgb.predict_proba(X_mod_test)[:,1]
    sub['isFraud'] = pred
    sub.to_csv(name, index=False)

In [50]:
%%time
submission2(model='xgboost',name = 'sub_xgb_3')

(506691,)         TransactionAmt  ProductCD  card1  card2  card3  card4  card5  card6  \
0            31.953125          4  10409  111.0  150.0      4  226.0      2   
1            49.000000          4   4272  111.0  150.0      4  226.0      2   
2           171.000000          4   4476  574.0  150.0      4  226.0      2   
3           285.000000          4  10989  360.0  150.0      4  166.0      2   
4            67.937500          4  18018  452.0  150.0      2  117.0      2   
...                ...        ...    ...    ...    ...    ...    ...    ...   
506686       94.687500          0  13832  375.0  185.0      2  224.0      2   
506687       12.171875          0   3154  408.0  185.0      2  224.0      2   
506688       49.000000          4  16661  490.0  150.0      4  226.0      2   
506689      202.000000          4  16621  516.0  150.0      2  224.0      2   
506690       24.343750          0   5713  168.0  144.0      4  147.0      1   

        addr1  addr2   dist1  P_emaildoma

[506691 rows x 318 columns]
Wall time: 6min 50s


In [35]:
%%time
submission2(model='lgbm',name='sub_lgbm_3')

(506691,)         TransactionAmt  ProductCD  card1  card2  card3  card4  card5  card6  \
0            31.953125          4  10409  111.0  150.0      4  226.0      2   
1            49.000000          4   4272  111.0  150.0      4  226.0      2   
2           171.000000          4   4476  574.0  150.0      4  226.0      2   
3           285.000000          4  10989  360.0  150.0      4  166.0      2   
4            67.937500          4  18018  452.0  150.0      2  117.0      2   
...                ...        ...    ...    ...    ...    ...    ...    ...   
506686       94.687500          0  13832  375.0  185.0      2  224.0      2   
506687       12.171875          0   3154  408.0  185.0      2  224.0      2   
506688       49.000000          4  16661  490.0  150.0      4  226.0      2   
506689      202.000000          4  16621  516.0  150.0      2  224.0      2   
506690       24.343750          0   5713  168.0  144.0      4  147.0      1   

        addr1  addr2   dist1  P_emaildoma

[506691 rows x 318 columns]
Wall time: 37 s


In [51]:
%%time
submission2(model='catboost',name='sub_cb_3')

(506691,)         TransactionAmt  ProductCD  card1  card2  card3  card4  card5  card6  \
0            31.953125          4  10409  111.0  150.0      4  226.0      2   
1            49.000000          4   4272  111.0  150.0      4  226.0      2   
2           171.000000          4   4476  574.0  150.0      4  226.0      2   
3           285.000000          4  10989  360.0  150.0      4  166.0      2   
4            67.937500          4  18018  452.0  150.0      2  117.0      2   
...                ...        ...    ...    ...    ...    ...    ...    ...   
506686       94.687500          0  13832  375.0  185.0      2  224.0      2   
506687       12.171875          0   3154  408.0  185.0      2  224.0      2   
506688       49.000000          4  16661  490.0  150.0      4  226.0      2   
506689      202.000000          4  16621  516.0  150.0      2  224.0      2   
506690       24.343750          0   5713  168.0  144.0      4  147.0      1   

        addr1  addr2   dist1  P_emaildoma

[506691 rows x 318 columns]
0:	learn: 0.5362906	total: 8.4s	remaining: 13m 51s
1:	learn: 0.4178554	total: 16.4s	remaining: 13m 22s
2:	learn: 0.3338323	total: 24.6s	remaining: 13m 14s
3:	learn: 0.2694033	total: 33.3s	remaining: 13m 18s
4:	learn: 0.2229336	total: 42.5s	remaining: 13m 27s
5:	learn: 0.1913739	total: 51.6s	remaining: 13m 27s
6:	learn: 0.1681659	total: 59.9s	remaining: 13m 15s
7:	learn: 0.1505858	total: 1m 8s	remaining: 13m 9s
8:	learn: 0.1385362	total: 1m 17s	remaining: 12m 59s
9:	learn: 0.1294917	total: 1m 25s	remaining: 12m 47s
10:	learn: 0.1210230	total: 1m 33s	remaining: 12m 34s
11:	learn: 0.1146431	total: 1m 41s	remaining: 12m 22s
12:	learn: 0.1096092	total: 1m 49s	remaining: 12m 13s
13:	learn: 0.1060938	total: 1m 57s	remaining: 12m 4s
14:	learn: 0.1023590	total: 2m 6s	remaining: 11m 54s
15:	learn: 0.0992464	total: 2m 14s	remaining: 11m 45s
16:	learn: 0.0968124	total: 2m 22s	remaining: 11m 36s
17:	learn: 0.0948959	total: 2m 30s	remaining: 11m 27s
18:	learn: 0.0922096	t

### 4. Experiment

In [41]:
def submission3(name = "submission.csv",category_features = []):
    
    X_mod = X.copy()
    y_mod = y.copy()
    X_mod_test = X_test.copy()
    pred = np.zeros(X_mod_test.shape[0])
        
    lg = lgb.LGBMClassifier(max_depth = 15, num_iterations = 200, num_leaves = 100,
                            subsample = 0.8, init_score = 0.3, boosting_type = 'goss',
                            objective = 'binary', min_split_gain =0.3)
    lg.fit(X_mod,y_mod,feature_name='auto', categorical_feature = category_features)
    pred1 = lg.predict_proba(X_mod_test)[:,1]
    
    for header in category_features:
        X_mod[header] = X_mod[header].astype('category').cat.codes
        X_mod_test[header] = X_mod_test[header].astype('category').cat.codes
    clf = cb.CatBoostClassifier(learning_rate = 0.1, depth = 15, l2_leaf_reg = 3.5,
                            rsm = 0.8, subsample = 0.8, iterations = 100)
    cat_dims = [X_train.columns.get_loc(i) for i in category_features[:-1]] 
    clf.fit(X_mod,y_mod, cat_features = cat_dims)
    pred2 = clf.predict_proba(X_mod_test)[:,1]
    
    model_xgb = xgb.XGBClassifier(base_score = 0.3, learning_rate = 0.1, max_depth = 15, 
                              colsample_bytree = 0.8, subsample = 0.8, n_estimators = 100,
                              gamma = 0.3)
    model_xgb.fit(X_mod, y_mod)
    pred3 = model_xgb.predict_proba(X_mod_test)[:,1]
    sub['isFraud'] = pred1/3 + pred2/3 + pred3/3
    sub.to_csv(name, index=False)

In [42]:
%%time
submission3(name = "blend_submission.csv")

0:	learn: 0.5362906	total: 8.47s	remaining: 13m 58s
1:	learn: 0.4178554	total: 16.1s	remaining: 13m 10s
2:	learn: 0.3338323	total: 23.9s	remaining: 12m 52s
3:	learn: 0.2694033	total: 31.7s	remaining: 12m 40s
4:	learn: 0.2229336	total: 39.6s	remaining: 12m 31s
5:	learn: 0.1913739	total: 47.3s	remaining: 12m 20s
6:	learn: 0.1681659	total: 55s	remaining: 12m 10s
7:	learn: 0.1505858	total: 1m 2s	remaining: 12m 1s
8:	learn: 0.1385362	total: 1m 10s	remaining: 11m 49s
9:	learn: 0.1294917	total: 1m 17s	remaining: 11m 40s
10:	learn: 0.1210230	total: 1m 25s	remaining: 11m 30s
11:	learn: 0.1146431	total: 1m 32s	remaining: 11m 21s
12:	learn: 0.1096092	total: 1m 40s	remaining: 11m 11s
13:	learn: 0.1060938	total: 1m 47s	remaining: 11m 2s
14:	learn: 0.1023590	total: 1m 55s	remaining: 10m 54s
15:	learn: 0.0992464	total: 2m 3s	remaining: 10m 47s
16:	learn: 0.0968124	total: 2m 11s	remaining: 10m 40s
17:	learn: 0.0948959	total: 2m 19s	remaining: 10m 33s
18:	learn: 0.0922096	total: 2m 27s	remaining: 10m 2

### 3. Validation

#### LightGBM

In [39]:
lg = lgb.LGBMClassifier(max_depth = 15, num_iterations = 200, num_leaves = 100,
                        subsample = 0.8, init_score = 0.3, boosting_type = 'goss',
                        objective = 'binary', min_split_gain =0.3)
lg.fit(X_train, y_train)
auc(lg, X_train, X_val)

0.9567104049084065

#### CatBoost

In [43]:
clf = cb.CatBoostClassifier(learning_rate = 0.1, depth = 15, l2_leaf_reg = 3.5,
                            rsm = 0.8, subsample = 0.8, iterations = 100)
clf.fit(X_train,y_train)
auc_test_cb = auc(clf, X_train, X_val)
print(auc_test_cb)

0:	learn: 0.5376578	total: 6.81s	remaining: 11m 13s
1:	learn: 0.4192691	total: 13.5s	remaining: 11m 3s
2:	learn: 0.3345935	total: 20.1s	remaining: 10m 51s
3:	learn: 0.2712698	total: 26.7s	remaining: 10m 41s
4:	learn: 0.2270546	total: 33.5s	remaining: 10m 35s
5:	learn: 0.1943363	total: 40.4s	remaining: 10m 32s
6:	learn: 0.1702373	total: 47.4s	remaining: 10m 29s
7:	learn: 0.1527631	total: 54.3s	remaining: 10m 24s
8:	learn: 0.1410451	total: 1m	remaining: 10m 14s
9:	learn: 0.1295888	total: 1m 7s	remaining: 10m 6s
10:	learn: 0.1207734	total: 1m 13s	remaining: 9m 58s
11:	learn: 0.1132743	total: 1m 20s	remaining: 9m 52s
12:	learn: 0.1080681	total: 1m 27s	remaining: 9m 45s
13:	learn: 0.1035153	total: 1m 34s	remaining: 9m 40s
14:	learn: 0.0995281	total: 1m 41s	remaining: 9m 35s
15:	learn: 0.0962255	total: 1m 48s	remaining: 9m 29s
16:	learn: 0.0933205	total: 1m 55s	remaining: 9m 24s
17:	learn: 0.0916386	total: 2m 2s	remaining: 9m 17s
18:	learn: 0.0895774	total: 2m 9s	remaining: 9m 11s
19:	learn:

#### XGBoost

In [44]:
model = xgb.XGBClassifier(base_score = 0.3, learning_rate = 0.1, max_depth = 15, 
                          colsample_bytree = 0.8, subsample = 0.8, n_estimators = 100,
                          gamma = 0.3)
model.fit(X_train, y_train)

auc(model, X_train, X_val)

0.9694246505781672